In [43]:
# * represent 0 or more other numbers
# @ represent 1 or more other numbers

# disallow if contain any 3*1@2, unless there is (-2)*3*1(-1)(2)
# disallow if contain any 1*3@2, unless there is 5*1*342


In [44]:
import itertools

In [49]:
def count(k):
    cnt = 0

    for seq in itertools.permutations(range(k)):
        pos = {k:i for i,k in enumerate(seq)}
        flag = False
        for i in range(k-2):
            a,b,c,d,e = i,i+1,i+2,i+3,i+4
            if pos[c] < pos[a] and pos[a] < pos[b]-1:
                if i >= 2 and pos[a-2] < pos[c] and pos[a]+2 == pos[a-1]+1 == pos[b]:
                    pass
                else:
                    flag = True
            if pos[a] < pos[c] and pos[c] < pos[b]-1:
                if i < k-4 and pos[e] < pos[a] and pos[c]+2 == pos[d]+1 == pos[b]:
                    pass
                else:
                    flag = True
        if flag:
            cnt += 1
        else:
            print(seq)
#     print(cnt)
    return cnt

In [50]:
count(5)

(0, 1, 2, 3, 4)
(0, 1, 2, 4, 3)
(0, 1, 3, 2, 4)
(0, 1, 4, 2, 3)
(0, 1, 4, 3, 2)
(0, 2, 1, 3, 4)
(0, 2, 1, 4, 3)
(0, 3, 1, 2, 4)
(0, 3, 2, 1, 4)
(0, 3, 4, 1, 2)
(0, 3, 4, 2, 1)
(0, 4, 1, 2, 3)
(0, 4, 1, 3, 2)
(0, 4, 2, 1, 3)
(0, 4, 3, 1, 2)
(0, 4, 3, 2, 1)
(1, 0, 2, 3, 4)
(1, 0, 2, 4, 3)
(1, 0, 3, 2, 4)
(1, 0, 4, 2, 3)
(1, 0, 4, 3, 2)
(1, 2, 0, 3, 4)
(1, 2, 0, 4, 3)
(1, 2, 3, 0, 4)
(1, 2, 3, 4, 0)
(1, 2, 4, 3, 0)
(1, 3, 2, 0, 4)
(1, 3, 2, 4, 0)
(1, 4, 0, 2, 3)
(1, 4, 0, 3, 2)
(1, 4, 2, 3, 0)
(1, 4, 3, 2, 0)
(2, 0, 1, 3, 4)
(2, 0, 1, 4, 3)
(2, 1, 0, 3, 4)
(2, 1, 0, 4, 3)
(2, 1, 3, 0, 4)
(2, 1, 3, 4, 0)
(2, 1, 4, 3, 0)
(2, 3, 0, 1, 4)
(2, 3, 1, 0, 4)
(2, 3, 1, 4, 0)
(2, 3, 4, 0, 1)
(2, 3, 4, 1, 0)
(2, 4, 3, 0, 1)
(2, 4, 3, 1, 0)
(3, 0, 1, 2, 4)
(3, 0, 2, 1, 4)
(3, 0, 4, 1, 2)
(3, 0, 4, 2, 1)
(3, 1, 2, 0, 4)
(3, 1, 2, 4, 0)
(3, 2, 0, 1, 4)
(3, 2, 1, 0, 4)
(3, 2, 1, 4, 0)
(3, 2, 4, 0, 1)
(3, 2, 4, 1, 0)
(3, 4, 0, 1, 2)
(3, 4, 0, 2, 1)
(3, 4, 1, 2, 0)
(3, 4, 2, 0, 1)
(3, 4, 2, 1, 0)
(4, 0, 1

42